In [6]:
# Auto-reload modules before executing code - great for development!
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('.')  # Add parent directory to path
from utils.tree_utils import ChristmasTree

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# Now you can use ChristmasTree here!
# Example:
tree = ChristmasTree(center_x='1.0', center_y='2.0', angle='45')
print(f"Tree created at ({tree.center_x}, {tree.center_y}) with angle {tree.angle}°")

Tree created at (1.0, 2.0) with angle 45°
